In [1]:
import os
import spacy

In [14]:
class InputDirectory:
    def __set__(self, instance, value):
        if not os.path.exists(value):
            raise FileNotFoundError('Директория не найдена')
        self.value = value

    def __get__(self, instance, owner):
        return self.value
    
class OutputDirectory:
    forbidden_symbols = [':', '?', '*', '\\', '<', '>', '|', '/', '"',] # символы, которых не должно быть в названии

    def __set__(self, instance, value):
        for symbol in self.forbidden_symbols:
          if symbol in value:
            raise ValueError('В названии содержаться недопустимые символы')
        self.value = value

    def __get__(self, instance, owner):
        return self.value
    
    

In [15]:
class CorpusCreator:
    input_dir = InputDirectory()
    output_dir = OutputDirectory()
    def __init__(self, input_dir, output_dir):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.nlp = spacy.load("en_core_web_sm")
    
    def openfile(self, file_path):
        try:
          with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()  
        except UnicodeDecodeError: # если не в utf-8
          with open('errors.txt', 'a', encoding='utf-8') as l: # сообщение об ошибке в отдельный файл
            l.write(f'Файл {file_path} не открылся из-за ошибки кодировки')
          return None # файл пропускается
        
    def parse(self, text):
        doc = self.nlp(text)
        return doc
    
    def writefile(self, output_path, doc):
        with open(output_path, 'w', encoding='utf-8') as f:
            for w in doc:
              f.write(f'{w.text}\t{w.pos_}\n') # записываю только слово и его часть речи

    def process(self):
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

        for root, dirs, files in os.walk(self.input_dir):
            for file in files:
                if file.endswith('.txt'): # чтобы попадали только текстовые файлы
                    file_path = os.path.join(root, file)
                    text = self.openfile(file_path)

                    if text is not None:
                        doc = self.parse(text)
                        output_file_name = os.path.splitext(file)[0] + '.conllu' # файл называется так же как раньше, только теперь .conllu
                        output_path = os.path.join(self.output_dir, output_file_name)
                        self.writefile(output_path, doc)
      
        
    


In [ ]:
corpus_creator = CorpusCreator('texts', 'textss')
corpus_creator.process()